In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT

datapath = 'predict_probs_resnet50_short.h5'
BATCHES_COUNT = 12091

In [2]:
store = pd.HDFStore(datapath)
print('{} tables in storage'.format(len(store.keys())))

1 tables in storage


In [15]:
# assert len(store.keys()) == BATCHES_COUNT

preds = []
for key in tqdm(store.keys()): 
    table = store.select(key)
    table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    assert table.shape[0] % 10 == 0
    for i in range(table.shape[0] / 10):
        product_id = table.pr_id.iloc[10*i]
        id_ = int(table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum().idxmax()) + 1
        cat = LABEL_TO_CAT[id_]
        preds.append((int(product_id), cat))

preds_df = pd.DataFrame(data=preds, columns=['_id','category_id'])
preds_df.to_csv('preds_resnet50.csv', index=False)

100%|██████████| 1/1 [00:00<00:00,  2.01it/s]
